In [61]:
import pyomo.environ as pyo
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [62]:
def load_data(file_name):
    file_path = Path(file_name).resolve()
    excel_file = pd.read_excel(file_path, sheet_name=None)
    sheet_dicts = {
        sheet_name: df.to_dict(orient="records")
        for sheet_name, df in excel_file.items()
    }
    # Assign each to a variable
    step1_data = sheet_dicts.get("Step 1")
    step2_data = sheet_dicts.get("Step 2")
    return sheet_dicts, step1_data, step2_data

def split_vendor_data(data):
    vendor_names = [entry['vendor'] for entry in data]
    
    production_dict = {entry['vendor']: entry['production'] for entry in data}
    cost_dict = {entry['vendor']: entry['cost'] for entry in data}
    
    return vendor_names, production_dict, cost_dict

In [ ]:
#TODO: think about units
#TODO: build it as a class
#TODO: pyomo set
def optimize(vendors, production, cost, demand):
    model = pyo.ConcreteModel(doc="flowsheet Optimization Model")
    steps = list(vendors.keys())
    index_set = [(g, o) for g in steps for o in vendors[g]]
    # Define binary decision variable
    model.x = pyo.Var(index_set, domain=pyo.Binary)

    # Constraint: select exactly one option per group
    def one_per_group_rule(model, g):
        return sum(model.x[g, o] for o in vendors[g]) == 1
    model.one_per_group = pyo.Constraint(steps, rule=one_per_group_rule)
    
    # Constraint: production level
    def prod_constraint(model):
        return sum(production[g][o] * model.x[g, o] for g in production for o in production[g]) >= demand
    model.prod_level = pyo.Constraint(rule=prod_constraint)

    # Objective: minimize cost
    def budget_constraint(model):
        return sum(production[g][o] * model.x[g, o] for g in production for o in production[g])
    model.obj = pyo.Objective(rule=budget_constraint, sense = pyo.minimize)

    
    opt = pyo.SolverFactory("gurobi")
    opt_success = opt.solve(model)

    model.pprint()


In [64]:
sheet_dicts, step1_data, step2_data = load_data("test-data.xlsx")
vendor_names1, production1, cost1 = split_vendor_data(step1_data)
vendor_names2, production2, cost2 = split_vendor_data(step2_data)
vendors = {"Step 1":vendor_names1, "Step 2":vendor_names2}
production = {"Step 1":production1, "Step 2":production2}
cost = {"Step 1":cost1, "Step 2":cost2}
print(vendors)
optimize(vendors, production, cost, 200)

{'Step 1': ['A', 'B'], 'Step 2': ['A', 'B', 'C', 'D']}
flowsheet Optimization Model

    1 Var Declarations
        x : Size=6, Index={('Step 1', 'A'), ('Step 1', 'B'), ('Step 2', 'A'), ('Step 2', 'B'), ('Step 2', 'C'), ('Step 2', 'D')}
            Key             : Lower : Value : Upper : Fixed : Stale : Domain
            ('Step 1', 'A') :     0 :   1.0 :     1 : False : False : Binary
            ('Step 1', 'B') :     0 :   0.0 :     1 : False : False : Binary
            ('Step 2', 'A') :     0 :   1.0 :     1 : False : False : Binary
            ('Step 2', 'B') :     0 :   0.0 :     1 : False : False : Binary
            ('Step 2', 'C') :     0 :   0.0 :     1 : False : False : Binary
            ('Step 2', 'D') :     0 :   0.0 :     1 : False : False : Binary

    1 Objective Declarations
        obj : Size=1, Index=None, Active=True
            Key  : Active : Sense    : Expression
            None :   True : minimize : 100*x[Step 1,A] + 150*x[Step 1,B] + 100*x[Step 2,A] + 150*x